In [87]:
from dask.distributed import Client

client = Client('10.65.18.58:8786')

/Users/jryu/.pyenv/versions/3.8.2/lib/python3.8/site-packages/distributed/client.py:1136: VersionMismatchWarning: Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| distributed | 2021.01.1 | 2021.02.0 | 2021.02.0 |
| lz4         | 3.1.3     | 3.1.1     | 3.1.1     |
| msgpack     | 1.0.2     | 1.0.0     | 1.0.0     |
| numpy       | 1.20.1    | 1.19.4    | 1.19.4    |
+-------------+-----------+-----------+-----------+
Notes: 
-  msgpack: Variation is ok, as long as everything is above 0.6
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [88]:
print(client)
client

<Client: 'tcp://172.18.0.6:8786' processes=3 threads=12, memory=49.97 GB>


Client Scheduler: tcp://10.65.18.58:8786 Dashboard: http://10.65.18.58:8787/status,Cluster Workers: 3 Cores: 12 Memory: 49.97 GB


In [89]:
import dask.dataframe as dd
import dask.bytes as db
import datetime
import pandas as pd
import math
import boto3
import json

In [90]:
import os
import sys
module_path = os.path.abspath(os.path.join('../flows'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [91]:
import tiles_pb2
from common import to_normalized_time, get_storage_option

In [92]:
client.upload_file('../flows/tiles_pb2.py')
client.upload_file('../flows/common.py')

{'tcp://10.0.33.32:45581': {'status': 'OK'},
 'tcp://10.0.33.33:38867': {'status': 'OK'},
 'tcp://10.0.33.5:41969': {'status': 'OK'}}

In [93]:
# Configs

source = {
    'endpoint_url': 'http://10.65.18.73:9000',
    'region_name':'us-east-1',
    'key': 'foobar',
    'secret': 'foobarbaz',
    'bucket': 'test'
}

dest = {
    'endpoint_url': 'http://10.65.18.73:9000',
    'region_name': 'us-east-1',
    'key': 'foobar',
    'secret': 'foobarbaz',
    'bucket': 'mass-upload-test'
}

# This determines the number of bins(subtiles) per tile. Eg. Each tile has 4^6=4096 grid cells (subtiles) when LEVEL_DIFF is 6
# Tile (z, x, y) will have a sutbile where its zoom level is z + LEVEL_DIFF
# eg. Tile (9, 0, 0) will have (15, 0, 0) as a subtile with LEVEL_DIFF = 6
LEVEL_DIFF = 6
MIN_SUBTILE_PRECISION = LEVEL_DIFF # since (0,0,0) main tile wil have (LEVEL_DIFF, x, y) subtiles as its grid cells

# Note: We need to figure out the spatial resolution of a run output in advance. For some model, 15 precision is way too high. 
# For example, lpjml model covers the entire world in very coarse resolution and with 15 precision, it takes 1 hour to process and upload
# the tiles resulting 397395 tile files. (uploading takes most of the time ) 
# where it takes only a minitue with 10 precision. And having high precision tiles doesn't make 
# significant difference visually since underlying data itself is very coarse.
MAX_SUBTILE_PRECISION = 14

# Maximum zoom level for a main tile
MAX_ZOOM = MAX_SUBTILE_PRECISION - LEVEL_DIFF

# TODO: provide these as input parameters
model_id = 'geo-test-data'
run_id = 'test-run'
s_bucket = source['bucket']
parquet_path = f's3://{s_bucket}/geo-test-data.parquet'


In [94]:
# Read parquet files in as set of dataframes
df = dd.read_parquet(parquet_path,
    storage_options={
        'anon': False,
        'use_ssl': False,
        'key': source['key'],
        'secret': source['secret'],
        'client_kwargs':{
            'region_name': source['region_name'],
            'endpoint_url': source['endpoint_url']
        }
    }).repartition(npartitions = 100)
df.dtypes

timestamp      int64
country       object
admin1        object
admin2        object
admin3        object
lat          float64
lng          float64
feature       object
value        float64
dtype: object

In [95]:
# Temporal aggregation (compute for both sum and mean)
time_res = 'month'

columns = df.columns.tolist()
columns.remove('value')

t = dd.to_datetime(df['timestamp'], unit='s').apply(lambda x: to_normalized_time(x, time_res), meta=(None, 'int'))
temporal_df = df.assign(timestamp=t) \
                .groupby(columns)['value'].agg(['sum', 'mean'])
# Rename agg column names
temporal_df.columns = temporal_df.columns.str.replace('sum', 't_sum').str.replace('mean', 't_mean')
temporal_df = temporal_df.reset_index()

In [96]:
temporal_df.tail()

,timestamp,country,admin1,admin2,admin3,lat,lng,feature,t_sum,t_mean
35,1577836800,South Sudan,Upper Nile,Tonga,Malakal,9.550000,31.650000,feature2,20.0,20.0
36,1593561600,Ethiopia,Afar,Afar Zone 1,Dubti,11.733333,41.083333,feature2,300.0,300.0
37,1593561600,Ethiopia,Afar,Afar Zone 3,Gewane,10.159228,40.661596,feature2,30.0,30.0
38,1593561600,Ethiopia,Tigray,Debubawi,Enderta,13.500000,39.666667,feature2,700.0,700.0
39,1593561600,South Sudan,Upper Nile,Tonga,Malakal,9.550000,31.650000,feature2,10.0,10.0


In [97]:
from copy import copy
def compute_regional_aggregation(input_df, dest, time_res, model_id, run_id):
    df = copy(input_df)
    # Ranme columns
    df.columns = df.columns.str.replace('t_sum', 't_sum_s_sum').str.replace('t_mean', 't_mean_s_sum')
    df['s_count'] = 1
    df = df.reset_index()
    
    ## TODO: extract region columns instead of hard coding since some data may not have admin3
    regions_cols = ['country', 'admin1', 'admin2', 'admin3']
    
    # Region aggregation at the highest admin level
    df = df[['feature', 'timestamp', 't_sum_s_sum', 't_mean_s_sum', 's_count'] + regions_cols] \
        .groupby(['feature', 'timestamp'] + regions_cols) \
        .agg(['sum'])
    df.columns = df.columns.droplevel(1)
    df = df.reset_index()
    # persist the result in memory since this df is going to be used multiple times to compute for different regional levels
    df = client.persist(df)
    
    # Compute aggregation and save for all regional levels
    for level in range(len(regions_cols)): 
        save_df = copy(df)
        # Merge region columns to single region_id column. eg. ['Ethiopia', 'Afar'] -> ['Ethiopia|Afar']
        save_df['region_id'] = join_region_columns(save_df, level)
    
        # groupby feature and timestamp
        save_df = save_df[['feature', 'timestamp', 'region_id', 't_sum_s_sum', 't_mean_s_sum', 's_count']] \
            .groupby(['feature', 'timestamp']).agg(list)
        save_df = save_df.reset_index()
        save_df = save_df.apply(lambda x: save_regional_aggregation(x, dest, model_id, run_id, time_res, region_level=regions_cols[level]), 
                      axis=1, meta=(None, 'object'))
        save_df.compute()

In [98]:
compute_regional_aggregation(temporal_df, dest, time_res, model_id, run_id)